# Olympics Data Analysis With Pyspark Dataframe

# ---------------------------------------------------------------------------------------------

# Create a spark container by calling SparkSession. This step is necessary before doing anything

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StringType, IntegerType 

# create session in order to be capable of accessing all Spark API

In [4]:
spark=SparkSession.builder.appName('olympics analysis').getOrCreate()

# define data schema for athlete_events.csv file and noc_regions.csv file

In [5]:
schema = StructType() \
    .add("ID",IntegerType(),True) \
    .add("Name",StringType(),True) \
    .add("Sex",StringType(),True) \
    .add("Age",IntegerType(),True) \
    .add("Height",IntegerType(),True) \
    .add("Weight",IntegerType(),True)\
    .add("Team",StringType(),True) \
    .add("NOC",StringType(),True) \
    .add("Games",StringType(),True) \
    .add("Year",IntegerType(),True) \
    .add("Season",StringType(),True) \
    .add("City",StringType(),True) \
    .add("Sport",StringType(),True) \
    .add("Event",StringType(),True) \
    .add("Medal",StringType(),True)

noc_schema = StructType() \
    .add("NOC",StringType(),True) \
    .add("region",StringType(),True) \
    .add("notes",StringType(),True) 

# read athlete_events.csv and noc_regions.csv files with defined schema into Spark DataFrame

In [6]:
df= spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("C:/ajaysir/P2/P2/athlete_events.csv")

df_noc= spark.read.format("csv") \
      .option("header", True) \
      .schema(noc_schema) \
      .load("C:/ajaysir/P2/P2/noc_regions.csv")

# show dataFrame's schema

In [7]:
df.printSchema()
df_noc.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Games: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Season: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)

root
 |-- NOC: string (nullable = true)
 |-- region: string (nullable = true)
 |-- notes: string (nullable = true)



# count number of rows in dataFrame 'df'

In [8]:
num_rows = df.count()
print("number of rows: ", num_rows)

number of rows:  271116


# show 10 rows of DataFrame 'df'

In [9]:
df.show(10)

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+-------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|        Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+-------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|   Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|         Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|  null|  null|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|     Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|  null|  null|Denmark/Sweden|DEN|1900 Summer|1900|Summer|      Paris|   Tug-Of-War|Tug-Of-War Me

# show statistic of the data

In [14]:
df.describe('age').show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|            261642|
|   mean|25.556898357297374|
| stddev| 6.393560847035813|
|    min|                10|
|    max|                97|
+-------+------------------+



# creating a temporary view of 'df' and 'df_noc' as 'athlete' and 'noc'

In [15]:
df.createOrReplaceTempView("athlete")
df_noc.createOrReplaceTempView("noc")

# 


# Querying with SQL

# ---------------------------------------------------------------

# 1) Write a Query to Count Female participants in year 2004 from India.

In [16]:
query="SELECT COUNT(*)  FROM athlete \
       WHERE Year='2004' and \
       Sex='F' and NOC in ('IND')"
spark.sql(query).show(10,truncate=False)

+--------+
|count(1)|
+--------+
|30      |
+--------+



# 2) Write a Query to fetch top 10 youngest athletes to ever compete in the Games.

In [19]:
query="SELECT name, age, city,Sport \
       FROM athlete \
       where Age is not null \
       Group By Name, age,City,sport ORDER BY age"
spark.sql(query).show(10,truncate=False)

+-----------------------------------------------+---+----------------------+--------------+
|name                                           |age|city                  |Sport         |
+-----------------------------------------------+---+----------------------+--------------+
|Dimitrios Loundras                             |10 |Athina                |Gymnastics    |
|Megan Olwen Devenish Taylor (-Mandeville-Ellis)|11 |Lake Placid           |Figure Skating|
|Marcelle Matthews                              |11 |Squaw Valley          |Figure Skating|
|Sonja Henie (-Topping- -Gardiner- -Onstad)     |11 |Chamonix              |Figure Skating|
|Beatrice Hutiu                                 |11 |Grenoble              |Figure Skating|
|Etsuko Inada                                   |11 |Garmisch-Partenkirchen|Figure Skating|
|Carlos Bienvenido Front Barrera                |11 |Barcelona             |Rowing        |
|Liana Vicens                                   |11 |Mexico City           |Swim

# 3) Write a Query to display the year in which women got started to enter into olympics.

In [20]:
query="SELECT Name,Year FROM athlete \
       WHERE Sex='F' \
       ORDER BY Year Asc"
spark.sql(query).show(1,truncate=False)

+-----------------------------+----+
|Name                         |Year|
+-----------------------------+----+
|Margaret Ives Abbott (-Dunne)|1900|
+-----------------------------+----+
only showing top 1 row



# 4) Write a Query to Count the number of male and female participants in each sport from 1896 to 2006.

In [21]:
query="SELECT s as Sport,Female_Count, Male_Count from \
       (SELECT Sport as s, Count(Name)as Female_Count FROM athlete WHERE Sex='F' GROUP BY Sport) \
       join \
       (SELECT Sport as p, Count(Name)as Male_Count FROM athlete WHERE Sex='M' GROUP BY Sport) \
       on s=p"
spark.sql(query).show(10,truncate=False)

+-------------+------------+----------+
|Sport        |Female_Count|Male_Count|
+-------------+------------+----------+
|Gymnastics   |9129        |17578     |
|Tennis       |1178        |1684      |
|Boxing       |72          |5975      |
|Ice Hockey   |754         |4762      |
|Golf         |70          |177       |
|Rowing       |2193        |8402      |
|Judo         |1093        |2708      |
|Sailing      |926         |5660      |
|Swimming     |9850        |13345     |
|Alpine Skiing|3398        |5431      |
+-------------+------------+----------+
only showing top 10 rows



# 5) Write a Query to Count the no of Female Participants from each country in every year.

In [23]:
query="SELECT Year,NOC as Country, Count(*) as Total_Female_Count \
       FROM athlete WHERE Sex='F' \
       GROUP BY NOC,Year order by Year"
spark.sql(query).show(10,truncate=False)

+----+-------+------------------+
|Year|Country|Total_Female_Count|
+----+-------+------------------+
|1900|USA    |9                 |
|1900|GBR    |2                 |
|1900|FRA    |16                |
|1900|SUI    |3                 |
|1900|ITA    |1                 |
|1900|BOH    |2                 |
|1904|USA    |16                |
|1906|GRE    |9                 |
|1906|FRA    |2                 |
|1908|GBR    |42                |
+----+-------+------------------+
only showing top 10 rows



# 6) Write a Query to fetch how many Indians participated in every year.

In [24]:
query="SELECT Year,Count(*) as Total_Participants \
       FROM athlete \
       WHERE NOC='IND' \
       GROUP BY Year \
       ORDER BY Year Asc"
spark.sql(query).show(10,truncate=False)

+----+------------------+
|Year|Total_Participants|
+----+------------------+
|1900|5                 |
|1920|7                 |
|1924|31                |
|1928|24                |
|1932|26                |
|1936|30                |
|1948|89                |
|1952|94                |
|1956|79                |
|1960|49                |
+----+------------------+
only showing top 10 rows



# 7) Write a Query to display distinct Sports in olympics.

In [71]:
query="select distinct sport \
       from athlete"
spark.sql(query).show(10,truncate=False)

+----------+
|sport     |
+----------+
|Gymnastics|
|Tennis    |
|Boxing    |
|Ice Hockey|
|Golf      |
|Rowing    |
|Judo      |
|Softball  |
|Sailing   |
|Swimming  |
+----------+
only showing top 10 rows



# 8) Write a Query to display no of females that have participated in olympics.

In [26]:
query="select year,count(sex) as Females \
       from athlete \
       where sex='F' \
       group by sex,year \
       order by year"
spark.sql(query).show(10,truncate=False)

+----+-------+
|year|Females|
+----+-------+
|1900|33     |
|1904|16     |
|1906|11     |
|1908|47     |
|1912|87     |
|1920|134    |
|1924|261    |
|1928|437    |
|1932|369    |
|1936|549    |
+----+-------+
only showing top 10 rows



# 9) Write a Query to display how many gold medals India won in each year.

In [27]:
query="select year,count(medal) as Gold \
       from athlete \
       where team='India' and medal='Gold' \
       group by year \
       order by year"
spark.sql(query).show(10,truncate=False)

+----+----+
|year|Gold|
+----+----+
|1924|7   |
|1928|14  |
|1932|15  |
|1936|19  |
|1948|20  |
|1952|14  |
|1956|17  |
|1964|15  |
|1980|16  |
|2008|1   |
+----+----+



# 10) Write a Query to count the number of participants in each sport in year 2016.

In [34]:
query=query="select s as sport,F,M \
             from (select sport as s,year,count(sex) as F \
                   from athlete \
                   where sex='F' and year='2016' \
                   group by sport,year) p \
               full join \
                 (select sport as sp,year,count(sex) as M \
                  from athlete where sex='M' and year='2016' \
                  group by sport,year) \
               on s=sp"
spark.sql(query).show(10,truncate=False) 

+-------------+---+---+
|sport        |F  |M  |
+-------------+---+---+
|Gymnastics   |383|478|
|Tennis       |143|143|
|Boxing       |36 |247|
|Golf         |60 |60 |
|Rowing       |219|331|
|Judo         |152|237|
|Sailing      |163|217|
|Swimming     |761|807|
|Basketball   |140|141|
|Equestrianism|131|224|
+-------------+---+---+
only showing top 10 rows



# 11) Write a Query to dispaly details of Indian players and medals they received.

In [70]:
query="select ROW_NUMBER() OVER(order by Total desc,Gold desc,Silver desc,Bronze desc) AS Rank,Name,\
              Sex,Total as Total_Medals,IFNULL(Gold,0) as Gold,IFNULL(Silver,0) as Silver,IFNULL(Bronze,0) as Bronze,Sport, \
              Team as Country \
       from (((select Sport,Sex,Name,Team,ID as fid,count(Medal) as Total from athlete \
               where Medal='Gold'or Medal='Silver' or Medal='Bronze' \
               group by ID,Name,Sport,Sex,Team) a0 \
            left join \
               (select ID,count(Medal) as Gold \
               from athlete \
               where Medal='Gold' \
               group by ID) a1 \
            on fid=a1.ID) p \
            left join \
                (select ID,count(Medal) as Silver \
                from athlete \
                where Medal='Silver' \
                group by ID) a2 \
            on fid=a2.ID) q \
            left join \
                 (select ID,count(Medal) as Bronze \
                 from athlete \
                 where Medal='Bronze' \
                 group by ID) a3 \
            on fid=a3.ID \
        where Team='India' \
        order by Rank"
spark.sql(query).show(10,truncate=False)

+----+-----------------------------+---+------------+----+------+------+------+-------+
|Rank|Name                         |Sex|Total_Medals|Gold|Silver|Bronze|Sport |Country|
+----+-----------------------------+---+------------+----+------+------+------+-------+
|1   |Udham Singh Kular            |M  |4           |3   |1     |0     |Hockey|India  |
|2   |Leslie Walter Claudius       |M  |4           |3   |1     |0     |Hockey|India  |
|3   |Balbir Singh Dosanjh- Sr.    |M  |3           |3   |0     |0     |Hockey|India  |
|4   |Dhyan Chand Bais             |M  |3           |3   |0     |0     |Hockey|India  |
|5   |Ranganathan Francis          |M  |3           |3   |0     |0     |Hockey|India  |
|6   |Richard James Allen          |M  |3           |3   |0     |0     |Hockey|India  |
|7   |Randhir Singh Gentle         |M  |3           |3   |0     |0     |Hockey|India  |
|8   |Shankar Pillay Laxman        |M  |3           |2   |1     |0     |Hockey|India  |
|9   |"Victor John ""V. J."" Pet

# 12) write a Query to display distinct player their country from India.

In [67]:
query="\
select distinct Name \
from athlete \
where Team='India'"
spark.sql(query).show(10,truncate=False)

+-----------------------+
|Name                   |
+-----------------------+
|Dattu Baban Bhokanal   |
|Keshav Chandra Dutt    |
|Monika Malik           |
|Dilip Mitra            |
|Rekha Mundphan (-Bhide)|
|Ajmer Singh            |
|Shaukat Ali            |
|Bimal Chandra          |
|Sakshi Malik           |
|Darshan Singh Kular    |
+-----------------------+
only showing top 10 rows



# 13) Write a Query to display max weight of all time in olympics.

In [66]:
query="\
select Name,max(weight) as weight,Team as Country,Sport \
from athlete \
group by Name,Team,Sport \
order by weight desc"
spark.sql(query).show(10,truncate=False)

+---------------------------------+------+-------------+-------------+
|Name                             |weight|Country      |Sport        |
+---------------------------------+------+-------------+-------------+
|Ricardo Blas- Jr.                |214   |Guam         |Judo         |
|Aytami Ruano Vega                |198   |Spain        |Judo         |
|Marek Galiski                    |190   |Poland       |Wrestling    |
|"Christopher J. ""Chris"" Taylor"|182   |United States|Wrestling    |
|Valentyn Rusliakov               |180   |Ukraine      |Judo         |
|Leonel Wilfredo Ruz Carmona      |178   |Venezuela    |Judo         |
|Dmitry Yuryevich Nosov           |175   |Russia       |Judo         |
|Andrey Ivanovich Chemerkin       |170   |Russia       |Weightlifting|
|Janusz Wojnarowicz               |170   |Poland       |Judo         |
|Illie Daniel Natea               |170   |Romania      |Judo         |
+---------------------------------+------+-------------+-------------+
only s

# 14) Write a Query to display which country received most medals in 2004.

In [65]:
query="\
select Team as Country,count(Medal) as Total_Medals \
from athlete \
where Medal in ('Gold','Silver','Bronze') and Year=2004 \
group by Team"
spark.sql(query).show(10,truncate=False)

+--------------+------------+
|Country       |Total_Medals|
+--------------+------------+
|Russia        |189         |
|Paraguay      |17          |
|Chinese Taipei|9           |
|Sweden        |12          |
|Spain-2       |2           |
|Eritrea       |1           |
|China-1       |8           |
|Turkey        |11          |
|Switzerland-2 |2           |
|Germany       |147         |
+--------------+------------+
only showing top 10 rows



# 15) Write a Query to display in which sport India won most medals from 1896-2006.

In [64]:
query="\
select sport,count(medal) as Total_Medals \
from athlete \
where Team='India' and Medal in ('Gold','Silver','Bronze') \
group by sport \
order by Total_Medals desc"
spark.sql(query).show(10,truncate=False)

+-------------+------------+
|sport        |Total_Medals|
+-------------+------------+
|Hockey       |173         |
|Alpinism     |7           |
|Wrestling    |5           |
|Shooting     |4           |
|Athletics    |2           |
|Badminton    |2           |
|Boxing       |2           |
|Weightlifting|1           |
|Tennis       |1           |
+-------------+------------+



# 16) Write a Query to display count of male and female in each year.

In [63]:
query="\
select r.year as year,IFNULL(F,0) as Female,M as Male \
from ((select distinct year \
       from athlete) r \
   left join \
     (select year,count(sex) as F from athlete \
      where sex='F' \
      group by year) p \
   on r.year=p.year) \
   left join \
     (select year,count(sex) as M \
      from athlete \
      where sex='M' \
      group by year) q \
   on r.year=q.year \
order by year"
spark.sql(query).show(10,truncate=False)

+----+------+----+
|year|Female|Male|
+----+------+----+
|1896|0     |380 |
|1900|33    |1903|
|1904|16    |1285|
|1906|11    |1722|
|1908|47    |3054|
|1912|87    |3953|
|1920|134   |4158|
|1924|261   |5432|
|1928|437   |5137|
|1932|369   |2952|
+----+------+----+
only showing top 10 rows



# 17) Write a Query to display all years in which olympics held.

In [57]:
query="\
SELECT DISTINCT year \
FROM athlete \
ORDER BY  year"
spark.sql(query).show(10,truncate=False)

+----+
|year|
+----+
|1896|
|1900|
|1904|
|1906|
|1908|
|1912|
|1920|
|1924|
|1928|
|1932|
+----+
only showing top 10 rows



# 18) Write a Query to display all the Countries participated in olympics from 1896-2006.

In [55]:
query="\
SELECT DISTINCT region AS Country \
FROM noc \
ORDER BY  Country"
spark.sql(query).show(10,truncate=False)

+--------------+
|Country       |
+--------------+
|Afghanistan   |
|Albania       |
|Algeria       |
|American Samoa|
|Andorra       |
|Angola        |
|Antigua       |
|Argentina     |
|Armenia       |
|Aruba         |
+--------------+
only showing top 10 rows



# 19) Write a Query to find in which year India got most medals.

In [56]:
query="\
SELECT year, count(medal) AS Total_Medals \
FROM athlete \
WHERE medal IN ('Gold','Silver','Bronze') AND Team='India' \
GROUP BY  year \
ORDER BY  Total_Medals DESC" 
spark.sql(query).show(10,truncate=False)

+----+------------+
|year|Total_Medals|
+----+------------+
|1948|20          |
|1936|19          |
|1956|17          |
|1968|16          |
|1980|16          |
|1952|15          |
|1932|15          |
|1964|15          |
|1972|14          |
|1928|14          |
+----+------------+
only showing top 10 rows



# 20) Write a Query to find Total_Medals,Gold Medals,Silver Medals and Bronze of Top 100 olympians with their country Name and sport Name.

In [53]:
query="\
SELECT ROW_NUMBER() OVER(order by Total desc,Gold desc,Silver desc,Bronze desc) AS Rank, \
        Name, \
        Sex, \
        Total AS Total_Medals, \
        IFNULL(Gold,0) AS Gold, \
        IFNULL(Silver,0) AS Silver, \
        IFNULL(Bronze,0) AS Bronze, \
        Sport, \
        Team AS Country \
FROM (((SELECT Sport,Sex,Name,Team,ID AS fid,count(Medal) AS Total \
        FROM athlete \
        WHERE Medal='Gold'or Medal='Silver' OR Medal='Bronze' \
        GROUP BY ID,Name,Sport,Sex,Team) a0 \
     LEFT JOIN \
        (SELECT ID,count(Medal) AS Gold \
        FROM athlete \
        WHERE Medal='Gold' \
        GROUP BY  ID) a1 \
     ON fid=a1.ID) p \
     LEFT JOIN \
        (SELECT ID,count(Medal) AS Silver \
        FROM athlete \
        WHERE Medal='Silver' \
        GROUP BY  ID) a2 \
     ON fid=a2.ID) q \
     LEFT JOIN \
        (SELECT ID,count(Medal) AS Bronze \
        FROM athlete \
        WHERE Medal='Bronze' \
        GROUP BY  ID) a3 \
     ON fid=a3.ID \
ORDER BY  Rank"
spark.sql(query).show(10,truncate=False)

+----+---------------------------------------------------+---+------------+----+------+------+----------+-------------+
|Rank|Name                                               |Sex|Total_Medals|Gold|Silver|Bronze|Sport     |Country      |
+----+---------------------------------------------------+---+------------+----+------+------+----------+-------------+
|1   |Michael Fred Phelps- II                            |M  |28          |23  |3     |2     |Swimming  |United States|
|2   |Larysa Semenivna Latynina (Diriy-)                 |F  |18          |9   |5     |4     |Gymnastics|Soviet Union |
|3   |Nikolay Yefimovich Andrianov                       |M  |15          |7   |5     |3     |Gymnastics|Soviet Union |
|4   |Ole Einar Bjrndalen                                |M  |13          |8   |4     |1     |Biathlon  |Norway       |
|5   |Borys Anfiyanovych Shakhlin                        |M  |13          |7   |4     |2     |Gymnastics|Soviet Union |
|6   |Edoardo Mangiarotti               

# 21) Write a Query to find Total_Medals,Gold Medals,Silver Medals,Bronze Medals of every country in the year 1912 with their rank.

In [54]:
query="\
SELECT ROW_NUMBER() OVER(order by Total desc,Gold desc,Silver desc,Bronze desc) AS Rank, \
       Country, \
       Total AS Total_Medals, \
       IFNULL(Gold,0) AS Gold, \
       IFNULL(Silver,0) AS Silver, \
       IFNULL(Bronze,0) AS Bronze \
FROM (((SELECT Team AS Country,count(Medal) AS Total \
        FROM athlete \
        WHERE Medal IN ('Gold','Silver','Bronze') AND year=1912 \
        GROUP BY  Team) a0 \
     LEFT JOIN \
        (SELECT Team,count(Medal) AS Gold \
         FROM athlete \
         WHERE Medal='Gold' AND year=1912 \
         GROUP BY  Team) a1 \
     ON Country=a1.Team) p \
     LEFT JOIN    \
         (SELECT Team,count(Medal) AS Silver \
          FROM athlete \
          WHERE Medal='Silver' AND year=1912 \
          GROUP BY  Team) a2 \
     ON Country=a2.Team) q \
     LEFT JOIN \
          (SELECT Team,count(Medal) AS Bronze \
           FROM athlete \
           WHERE Medal='Bronze' AND year=1912 \
           GROUP BY  Team) a3 \
      ON Country=a3.Team  \
ORDER BY  Rank"
spark.sql(query).show(10,truncate=False)

+----+-------------+------------+----+------+------+
|Rank|Country      |Total_Medals|Gold|Silver|Bronze|
+----+-------------+------------+----+------+------+
|1   |Sweden       |153         |95  |38    |20    |
|2   |Great Britain|128         |36  |35    |57    |
|3   |United States|107         |46  |25    |36    |
|4   |Denmark      |74          |0   |46    |28    |
|5   |Norway       |55          |25  |6     |24    |
|6   |Finland      |52          |9   |33    |10    |
|7   |Germany      |39          |6   |19    |14    |
|8   |Hungary      |30          |10  |17    |3     |
|9   |Netherlands  |26          |0   |0     |26    |
|10  |Italy        |25          |22  |1     |2     |
+----+-------------+------------+----+------+------+
only showing top 10 rows

